<a href="https://colab.research.google.com/github/Ali-mohammadi-design/ML-practice/blob/main/Prompt_engineering_Fine_tunning_Medical_Text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [4]:
import openai
print(openai.__version__)

1.6.1


In [7]:
import os
API=os.environ['openAI_API_Key']

In [8]:
from openai import OpenAI
client= OpenAI(api_key=API)

We want to fine tune the medical report including the name of the disease and medical emergencies as well as their description.

 The very first step is data preprocessing or data cleaning

Data PreProcessing:

In [10]:
import pandas as pd
medical_reports=pd.read_csv('/content/reports_copy.csv')

In [11]:
medical_reports.head()

,medical_specialty,report
0,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
1,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...
2,Cardiovascular / Pulmonary,"2-D ECHOCARDIOGRAM,Multiple views of the heart..."
3,Cardiovascular / Pulmonary,"DESCRIPTION:,1. Normal cardiac chambers size...."
4,Cardiovascular / Pulmonary,"2-D STUDY,1. Mild aortic stenosis, widely calc..."


In [12]:
medical_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


Removing the nulls in the report section:

Note: We can use fillna to fill the nulls

In [13]:
medical_reports= medical_reports.dropna(subset=['report'])

In [14]:
medical_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


In the next step we have to split the data to train, test, and validation

In [15]:
medical_reports.groupby('medical_specialty').count()

,report
medical_specialty,
Cardiovascular / Pulmonary,371
Gastroenterology,224
Neurology,223
Radiology,273
Surgery,1088


Note: .groupby() would group all the specific parameters in a context (here is medical specialty). The we would seperate the specific amount of sample from each of the groups.

In [16]:
grouped_data= medical_reports.groupby('medical_specialty').sample(110, random_state=42)

In [17]:
grouped_data['medical_specialty'].value_counts()

Cardiovascular / Pulmonary    110
Gastroenterology              110
Neurology                     110
Radiology                     110
Surgery                       110
Name: medical_specialty, dtype: int64

Now in each group we have 110 data. For every group we could separate 10 data for validation and test (5 for validation and 5 for test).

In [18]:
val_test_data= grouped_data.groupby('medical_specialty').sample(10, random_state=42)

In [19]:
val_data= val_test_data.groupby('medical_specialty').head(5)
test_data= val_test_data.groupby('medical_specialty').tail(5)

Now we have to remove the test and validation data from the main data. We can use the .index.isin() function.

Index.isin(values, level=None)

Return a boolean array where the index values are in values.


note: ~ means not

note2: .index would derermine the position of the data in the list

In [20]:
train= grouped_data[~grouped_data.index.isin(val_test_data.index)]

In [21]:
len(train)

500

reviewing statistics and estimating the pricing!

In [22]:
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [23]:
import tiktoken

In [24]:
def num_tokens_from_string(string):
  encoding=tiktoken.get_encoding('cl100k_base')
  num_token=len(encoding.encode(string))
  return num_token

In [25]:
num_tokens_from_string("hello world, how are you?")

7

.Apply()

Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1). apply function to each column., the final return type is inferred from the return type of the applied function

In [26]:
report_length= train['report'].apply(num_tokens_from_string)

In [27]:
report_length.describe()

count     500.000000
mean      674.788000
std       427.809075
min        14.000000
25%       358.000000
50%       593.500000
75%       904.000000
max      3701.000000
Name: report, dtype: float64

In [28]:
number_of_tokens=report_length.sum()
number_of_tokens

337394

Thus, in total we have 337394 tokens!!!

in the next stage we would calculate the pricing

In [29]:
price_per_1000_Token_training= 0.008/1000

In [30]:
final_price=price_per_1000_Token_training*number_of_tokens
final_price

2.6991519999999998

In [31]:
train['medical_specialty'].unique()

array(['Cardiovascular / Pulmonary', 'Gastroenterology', 'Neurology',
       'Radiology', 'Surgery'], dtype=object)

NOte: To fine tune the LLM we need three parts.

system_message

User_message -----> report

Assistant-----> medical_specialty

In [32]:
system_prompt='given the medical description report, classufy it into one of these categories:Cardiovascular / Pulmonary,Gastroenterology, Neurology, Radiology,Surgery  '

In [33]:
print(system_prompt)

given the medical description report, classufy it into one of these categories:Cardiovascular / Pulmonary,Gastroenterology, Neurology, Radiology,Surgery  


Note: to fine tune the GPT we need a dictionarry with the following format:

sample_prompt={'messages':[{},{},{}]}


this is the dictionarry that includes a list of three dictionaries

In [34]:
sample_prompt={'messages':[{'role':'system', 'content': system_prompt},
 {'role':'user', 'content': train['report'].iloc[0]},
  {'role': 'assistant', 'content':train['medical_specialty'].iloc[0]}]}

In [35]:
print(sample_prompt)

{'messages': [{'role': 'system', 'content': 'given the medical description report, classufy it into one of these categories:Cardiovascular / Pulmonary,Gastroenterology, Neurology, Radiology,Surgery  '}, {'role': 'user', 'content': "PREOPERATIVE DIAGNOSES:,1.  Lumbar osteomyelitis.,2.  Need for durable central intravenous access.,POSTOPERATIVE DIAGNOSES:,1.  Lumbar osteomyelitis.,2.  Need for durable central intravenous access.,ANESTHESIA:,  General.,PROCEDURE:,  Placement of left subclavian 4-French Broviac catheter.,INDICATIONS:  ,The patient is a toddler admitted with a limp and back pain, who was eventually found on bone scan and septic workup to have probable osteomyelitis of the lumbar spine at disk areas.  The patient needs prolonged IV antibiotic therapy, but attempt at a PICC line failed.  She has exhausted most of her easy peripheral IV access routes and referral was made to the Pediatric Surgery Service for Broviac placement.  I met with the patient's mom.  With the help of a

In [36]:
def df_to_format(df):
  formatted_data=[]

  for index, row in df.iterrows():
    entry={'messages':[{'role':'system', 'content': system_prompt},
          {'role':'user', 'content': row['report']},
          {'role': 'assistant', 'content':row['medical_specialty']}]}
    formatted_data.append(entry)
  return formatted_data



In [37]:
data=df_to_format(train)

In [39]:
type(data)

list

in the next step we have to convert this list of dictionary to the JS file

In [41]:
import json

In [42]:
with open('fine_tuning_data.jsonl','w') as f:
  for entry in data:
    f.write(json.dumps(entry))
    f.write("\n")

In [44]:
val_data_dic=df_to_format(val_data)

In [46]:
with open('fine_tuning_data_val.jsonl','w') as f:
  for entry in val_data_dic:
    f.write(json.dumps(entry))
    f.write("\n")

Training data

In [48]:
file_upload_response= client.files.create(file=open('fine_tuning_data.jsonl', 'rb'), purpose='fine-tune')

In [49]:
file_upload_response

FileObject(id='file-8ebomr4y8glSovhbzZTsCRfG', bytes=1542122, created_at=1704298077, filename='fine_tuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [51]:
type(file_upload_response)

openai.types.file_object.FileObject

In [50]:
file_upload_response_val= client.files.create(file=open('fine_tuning_data_val.jsonl', 'rb'), purpose='fine-tune')

In [52]:
fine_tuning_response= client.fine_tuning.jobs.create(training_file=file_upload_response.id,model='gpt-3.5-turbo',
                                                     hyperparameters={'n_epochs':1},
                                                     validation_file=file_upload_response_val.id)

In [53]:
fine_tuning_response

FineTuningJob(id='ftjob-mtZS7P3wKO8YJlvEXPr10Q3t', created_at=1704298599, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jAjbk43XSY8d16iQRdOI2ms9', result_files=[], status='validating_files', trained_tokens=None, training_file='file-8ebomr4y8glSovhbzZTsCRfG', validation_file='file-ojmrPjiZs9Ji9asoeLqzxeKH')

In [55]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-mtZS7P3wKO8YJlvEXPr10Q3t', created_at=1704298599, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jAjbk43XSY8d16iQRdOI2ms9', result_files=[], status='validating_files', trained_tokens=None, training_file='file-8ebomr4y8glSovhbzZTsCRfG', validation_file='file-ojmrPjiZs9Ji9asoeLqzxeKH')], object='list', has_more=False)

As is clear in the previous status, the finish at is on None status that shows that fine tuning is still runing. So, we should wait untile it is completely done!!

In [56]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-mtZS7P3wKO8YJlvEXPr10Q3t', created_at=1704298599, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jAjbk43XSY8d16iQRdOI2ms9', result_files=[], status='running', trained_tokens=None, training_file='file-8ebomr4y8glSovhbzZTsCRfG', validation_file='file-ojmrPjiZs9Ji9asoeLqzxeKH')], object='list', has_more=False)

In [57]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-mtZS7P3wKO8YJlvEXPr10Q3t', created_at=1704298599, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jAjbk43XSY8d16iQRdOI2ms9', result_files=[], status='running', trained_tokens=None, training_file='file-8ebomr4y8glSovhbzZTsCRfG', validation_file='file-ojmrPjiZs9Ji9asoeLqzxeKH')], object='list', has_more=False)

In [58]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-mtZS7P3wKO8YJlvEXPr10Q3t', created_at=1704298599, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal::8cyZgOnX', finished_at=1704300003, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-jAjbk43XSY8d16iQRdOI2ms9', result_files=['file-aqsaA7qe8wFjoEWaWXU7Na4x'], status='succeeded', trained_tokens=363094, training_file='file-8ebomr4y8glSovhbzZTsCRfG', validation_file='file-ojmrPjiZs9Ji9asoeLqzxeKH')], object='list', has_more=False)

Evaluating the fine_tuning

In [59]:
train_event= client.fine_tuning.jobs.list_events(fine_tuning_job_id='The Model code')

Testing the model

In [65]:
test_data.iloc[0]

medical_specialty                           Cardiovascular / Pulmonary
report               CLINICAL INDICATION:,  Chest pain.,INTERPRETAT...
Name: 2016, dtype: object

In [62]:
test_data['report'].iloc[0]

"CLINICAL INDICATION:,  Chest pain.,INTERPRETATION: , The patient received 14.9 mCi of Cardiolite for the rest portion of the study and 11.5 mCi of Cardiolite for the stress portion of the study.,The patient's baseline EKG was normal sinus rhythm.  The patient was stressed according to Bruce protocol by Dr. X.  Exercise test was supervised and interpreted by Dr. X.  Please see the separate report for stress portion of the study.,The myocardial perfusion SPECT study shows there is mild anteroseptal fixed defect seen, which is most likely secondary to soft tissue attenuation artifact.  There is, however, mild partially reversible perfusion defect seen, which is more pronounced in the stress images and short-axis view suggestive of minimal ischemia in the inferolateral wall.,The gated SPECT study shows normal wall motion and wall thickening with calculated left ventricular ejection fraction of 59%.,CONCLUSION:,1.  The exercise myocardial perfusion study shows possibility of mild ischemia 

In [72]:
completion= client.chat.completions.create(model='The model code on the OpenAI fine Tuning section', messages=[
    {'role':'system', 'content':system_prompt},
    {'role':'user', 'content':test_data['report'].iloc[0] }


])

BadRequestError: ignored

In [69]:
completion

ChatCompletion(id='chatcmpl-8czBijT146jMKR0qapRZ0n53l8e7C', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Cardiovascular / Pulmonary', role='assistant', function_call=None, tool_calls=None))], created=1704302362, model='ft:gpt-3.5-turbo-0613:personal::8cyZgOnX', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=290, total_tokens=296))

In [71]:
completion.choices[0].message

ChatCompletionMessage(content='Cardiovascular / Pulmonary', role='assistant', function_call=None, tool_calls=None)